In [119]:
import pandas as pd
import numpy as np
import ast
import re

In [102]:
#Load data
data = pd.read_csv('Press_Conferences.csv', index_col=0, converters={"answer": ast.literal_eval})
#data = pd.read_csv('Press_Conferences.csv')

In [103]:
# Checking data
for n, i in zip(data.iloc[19347:,]['question'],data.iloc[19347:,]['answer']):
    print(n,i)

MODERATOR ('jon rahm', ['it', 'could', 'go', 'two', 'ways', 'right', 'something', 'special', 'going', 'on', 'or', "i'm", 'just', 'running', 'out', 'of', 'luck', 'already', 'so', 'i', "don't", 'know', 'i', 'knew', 'it', 'would', 'be', 'a', 'special', 'week', 'given', "it's", 'my', 'birthday', 'today', 'so', "i'm", 'hoping', "it's", 'the', 'beginning', 'of', 'a', 'lot', 'of', 'good', 'things', 'to', 'come'])
MODERATOR ('jon rahm', ['yeah', 'on', 'i', 'hit', 'it', 'to', 'about', 'an', 'inch', 'and', 'justin', 'thomas', 'made', 'it', 'on', 'top', 'of', 'me', 'a', 'little', 'bit', 'of', 'revenge', 'on', 'that', 'hole', 'i', 'guess'])
MODERATOR ('jon rahm', ['nothing', 'special', 'what', 'can', 'we', 'do', 'go', 'home', 'have', 'dinner', 'and', 'sleep', 'i', 'mean', "there's", 'nothing', 'else', 'we', 'can', 'do', 'hopefully', 'because', 'of', 'being', 'the', 'week', 'that', 'it', 'is', 'and', "what's", 'going', 'on', 'in', 'the', 'world', 'i', 'think', 'all', 'celebrations', 'are', 'going',

In [104]:
print(len(data['question']))
print(data['question'][data['question'] == 'MODERATOR'].value_counts())

19352
question
MODERATOR    4023
Name: count, dtype: int64


We'll need to Review The Scraper loop to avoid questions beeing incorrectly classified as MODERATOR

In [121]:
data

,question,answer
date,,
"March 3, 2024",a little bit of a race to ...,"(rory mcilroy, [a, week, that, could, have, be..."
"March 3, 2024",you added this event added va...,"(rory mcilroy, [yeah, if, anything, i'm, just,..."
"March 3, 2024",unless you really think about ...,"(rory mcilroy, [i'm, really, happy, i, got, do..."
"March 3, 2024",with shane kind of in content...,"(rory mcilroy, [it, would, mean, a, lot, he, l..."
"February 29, 2024",what did you think of your ...,"(rory mcilroy, [pretty, solid, some, good, som..."
...,...,...
"November 10, 2020",MODERATOR,"(jon rahm, [it, could, go, two, ways, right, s..."
"November 10, 2020",MODERATOR,"(jon rahm, [yeah, on, i, hit, it, to, about, a..."
"November 10, 2020",MODERATOR,"(jon rahm, [nothing, special, what, can, we, d..."


In [120]:
#Transform questions using Regex to leave only text
data['question'] = data['question'].str.replace(r"[\['\],]", ' ', regex=True)

In [118]:
data.question

date
March 3, 2024        ['a', 'little', 'bit', 'of', 'a', 'race', 'to'...
March 3, 2024        ['you', 'added', 'this', 'event', 'added', 'va...
March 3, 2024        ['unless', 'you', 'really', 'think', 'about', ...
March 3, 2024        ['with', 'shane', 'kind', 'of', 'in', 'content...
February 29, 2024    ['what', 'did', 'you', 'think', 'of', 'your', ...
                                           ...                        
November 10, 2020                                            MODERATOR
November 10, 2020                                            MODERATOR
November 10, 2020                                            MODERATOR
November 10, 2020                                            MODERATOR
November 10, 2020                                            MODERATOR
Name: question, Length: 19352, dtype: object